In [1]:
import numpy as np
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
from scipy.stats import poisson
import statistics as st
import matplotlib.pyplot as plt

In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing

print(tf.__version__)

2.6.0


In [3]:
# Input sheet for model to train on
afc = pd.read_csv("/Users/abhijotsingh/Desktop/AtlassianDatathon/AFC/INPUT/afc_input.csv")
# Predicting upcoming matches sheet
# upcoming_afc = pd.read_csv("/Users/abhijotsingh/Desktop/AtlassianDatathon/upcoming_afc.csv")
# Predicting 3rd place match
upcoming_afc = pd.read_csv("/Users/abhijotsingh/Desktop/AtlassianDatathon/AFC/INPUT/thirdplace.csv")

In [4]:
# Regression model - Neural network for home_xG, away_xg
home_xg_df = afc[['home_score', 'H_attack', 'A_defence']]
away_xg_df = afc[['away_score', 'H_defence', 'A_attack']]

# Splitting dataset
train_homexG = home_xg_df.sample(frac=0.8, random_state = 0)
test_homexG = home_xg_df.drop(train_homexG.index)

train_awayxG = away_xg_df.sample(frac=0.8, random_state = 0)
test_awayxG = away_xg_df.drop(train_awayxG.index)

# For home_xG
train_features = train_homexG.copy()
test_features = test_homexG.copy()

train_labels = train_features.pop('home_score')
test_labels = test_features.pop('home_score')

# For away_xG
train_features2 = train_awayxG.copy()
test_features2 = test_awayxG.copy()

train_labels2 = train_features2.pop('away_score')
test_labels2 = test_features2.pop('away_score')

In [5]:
# Pre-processing, home / away
normalizer = preprocessing.Normalization(axis=-1)
normalizer.adapt(np.array(train_features))

normalizer2 = preprocessing.Normalization(axis=-1)
normalizer2.adapt(np.array(train_features2))

# Model functions, home / away
# home
linear_model = tf.keras.Sequential([
    normalizer,
    layers.Dense(units=1)
])
# away
linear_model2 = tf.keras.Sequential([
    normalizer2,
    layers.Dense(units=1)
])

def build_and_compile_model(norm):
  model = keras.Sequential([
      norm,
      layers.Dense(64, activation='relu'),
      layers.Dense(64, activation='relu'),
      layers.Dense(1)
  ])

  model.compile(loss='mean_absolute_error',
                optimizer=tf.keras.optimizers.Adam(0.0025))
  return model

# Building model, home / away
dnn_model = build_and_compile_model(normalizer)
dnn_model.summary()

dnn_model2 = build_and_compile_model(normalizer2)
dnn_model2.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
normalization (Normalization (None, 2)                 5         
_________________________________________________________________
dense_2 (Dense)              (None, 64)                192       
_________________________________________________________________
dense_3 (Dense)              (None, 64)                4160      
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 65        
Total params: 4,422
Trainable params: 4,417
Non-trainable params: 5
_________________________________________________________________
Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
normalization_1 (Normalizati (None, 2)                 5         
______________________________

In [6]:
%%time
history = dnn_model.fit(
    train_features, train_labels,
    validation_split=0.2,
    verbose=0, epochs=1000)

# training model home

CPU times: user 23.8 s, sys: 3.06 s, total: 26.8 s
Wall time: 20.4 s


In [7]:
%%time
history = dnn_model2.fit(
    train_features2, train_labels2,
    validation_split=0.2,
    verbose=0, epochs=1000)

# training model away

CPU times: user 23.5 s, sys: 3.18 s, total: 26.7 s
Wall time: 20 s


In [8]:
test_results = {}
test_results['dnn_model'] = dnn_model.evaluate(test_features, test_labels, verbose=0)
pd.DataFrame(test_results, index=['Mean absolute error [home_score]']).T

test_results2 = {}
test_results2['dnn_model2'] = dnn_model2.evaluate(test_features2, test_labels2, verbose=0)
pd.DataFrame(test_results2, index=['Mean absolute error [away_score]']).T


,Mean absolute error [away_score]
dnn_model2,0.913539


In [9]:
# Predicting home_xG and away_xG for upcoming fixtures
home_xg_features = upcoming_afc[['H_attack', 'A_defence']]
pred_home_xg = dnn_model.predict(home_xg_features).flatten()
pred_home_xg = [0 if i < 0 else i for i in pred_home_xg]

away_xg_features = upcoming_afc[['H_defence', 'A_attack']]
pred_away_xg = dnn_model2.predict(away_xg_features).flatten()
pred_away_xg = [0 if i < 0 else i for i in pred_away_xg]
pred_away_xg

[0.69554865, 1.0662184]

In [17]:
# Run the poisson results for the home team
home_outcome = []

for i in range(len(pred_home_xg)):
    expected = pred_home_xg[i] + 0.5
    poi = poisson.rvs(expected, size=10000)
    # Select most common outcome and append to the poisson outcome list
    common = st.mode(poi)
    home_outcome.append(common)

home_outcome

[0, 1]

In [18]:
# Run the poisson results for the away team
away_outcome = []

for i in range(len(pred_away_xg)):
    expected = pred_away_xg[i] + 0.5
    poi = poisson.rvs(expected, size=10000)
    # Select most common outcome and append to the poisson outcome list
    common = st.mode(poi)
    away_outcome.append(common)

away_outcome

[1, 1]

In [19]:
upcoming_afc["Poisson_H_xG"] = home_outcome
upcoming_afc["Poisson_A_xG"] = away_outcome
afc_result = upcoming_afc[['Home', 'Away', 'Poisson_H_xG', 'Poisson_A_xG']]

result = []
outcomes = ["H_WIN", "DRAW", "A_WIN"]
for i in range(len(away_outcome)):
    if upcoming_afc["Poisson_H_xG"][i] > upcoming_afc["Poisson_A_xG"][i]:
        result.append(outcomes[0])
    elif upcoming_afc["Poisson_H_xG"][i] == upcoming_afc["Poisson_A_xG"][i]:
        result.append(outcomes[1])
    else:
        result.append(outcomes[2])

afc_result["Results"] = result

H_Points = []
A_Points = []
for i in range(len(away_outcome)):
    if result[i] == "H_WIN":
        H_Points.append(3)
        A_Points.append(0)
    elif result[i] == "DRAW":
        H_Points.append(1)
        A_Points.append(1)
    else:
        H_Points.append(0)
        A_Points.append(3)
       
        
afc_result["H_Points"] = H_Points
afc_result["A_Points"] = A_Points
afc_result

,Home,Away,Poisson_H_xG,Poisson_A_xG,Results,H_Points,A_Points
0,South Korea,Japan,0,1,A_WIN,0,3
1,Japan,South Korea,1,1,DRAW,1,1


In [20]:
# Create Points Table
afc_home = afc_result.groupby("Home")["H_Points"].sum()
afc_away = afc_result.groupby("Away")["A_Points"].sum()
points = afc_home + afc_away
points

Home
Japan          4
South Korea    1
dtype: int64

In [ ]:
#afc_result.to_csv("/Users/abhijotsingh/Desktop/AtlassianDatathon/CAF/CAF_upcomingresults_dnn3.csv", index=False)
#points.to_csv("/Users/abhijotsingh/Desktop/AtlassianDatathon/CAF/CAF_POINTS_change.csv", index=False)